In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold

In [2]:
kaggle_data_folder = "/media/vincentwang/Backup/kaggle_data/ventilator-pressure-prediction"
output_folder = "../output/"

In [3]:
train = pd.read_csv(kaggle_data_folder + "/train.csv")

In [8]:
seed = 48
y = train['R'].astype(str)  + "__" + train['C'].astype(str)

In [11]:
y

0          20__50
1          20__50
2          20__50
3          20__50
4          20__50
            ...  
6035995    50__10
6035996    50__10
6035997    50__10
6035998    50__10
6035999    50__10
Length: 6036000, dtype: object

In [32]:
RC_train = train.groupby("id")[["R","C"]].last()
RC_train.reset_index(inplace=True)
RC_train["R_C"] = RC_train["R"].astype(str) + "_" + RC_train["C"].astype(str)

In [33]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
splits = cv.split(RC_train, RC_train['R_C'])

In [34]:
RC_train['fold'] = -1
for i,(train_idx, valid_idx) in enumerate(splits):
    print(i)
    RC_train.loc[valid_idx,"fold"] = i

0
1
2
3
4


In [35]:
# double check
# check id
RC_train.groupby("fold")["id"].unique().apply(len).sum() == len(train.id.unique())

True

In [38]:
# check R_C group
RC_train.groupby(["fold","R_C"]).size().groupby("fold").unique()

fold
0    [97120, 99328, 130976, 218832, 132160, 131024,...
1    [97120, 99328, 130976, 218832, 132160, 131024,...
2    [97120, 99328, 130976, 218832, 132160, 131024,...
3    [97120, 99328, 130976, 218832, 132160, 131024,...
4    [97120, 99328, 130976, 218832, 132160, 131024,...
dtype: object

In [40]:
id_fold_dict = dict(zip(RC_train["id"], RC_train["fold"]))

In [41]:
train['fold'] = train['id'].apply(lambda x: id_fold_dict[x])

In [44]:
import pickle
with open('../output/id_fold_dict.pickle', 'wb') as handle:
    pickle.dump(id_fold_dict, handle)
print("save id_fold_dict")

save id_fold_dict
